In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os, sys

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
import random
from time import time
import concurrent
from concurrent.futures import ProcessPoolExecutor
from consolidated_runs import run_sergio
from SERGIO.SERGIO.sergio import sergio
import parallel_utils

def new_expected_value_experiment(dataset_id, last_iteration=0, fixed_edge=False, add_edge=True, multiple_edges=False, clean='clean'):
    #print("Running edge finding experiment")
    gt_file = None
    if dataset_id == 1:
        gt_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Interaction_cID_4.txt'
        regs_path = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Regs_cID_4.txt'
        n_genes = 100
    elif dataset_id == 2:
        gt_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Interaction_cID_5.txt'
        regs_path = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Regs_cID_5.txt'
        n_genes = 400
    elif dataset_id == 3:
        gt_file = 'SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'
        regs_path = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt'
        n_genes = 1200
    reg_df = pd.read_csv(regs_path, header=None)
    master_regs = [int(m) for m in reg_df[0].values]
    true_pearson = pd.DataFrame()
    gt = pd.read_csv(gt_file, header=None)
    imp_dir = os.path.join(os.getcwd(), 'imputations')
    load_dir = os.path.join(imp_dir, f'DS{dataset_id}')
    ranks = []
    means = []

    experiment_dir = os.path.join(os.getcwd(), f'experiments/mean_diffs/DS{dataset_id}')
    if not os.path.exists(experiment_dir):
        os.makedirs(experiment_dir)
    experiment_file = os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment.csv")
    if fixed_edge:
        experiment_file = os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment_fixed_edge.csv")
    if os.path.exists(experiment_file) and last_iteration != 0:
        df = pd.read_csv(experiment_file)
        ranks = df['Rank'].values.tolist()
        means = df['Correlation'].values.tolist()

    #run_sergio(target_file, regs_path, dataset_id, file_extension='')

    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = []
        for iteration in range(last_iteration, 51):      
            file_extension = ''                          
            file_extension = f"_iter{iteration}"
            # Get number of genes to choose a target
            if iteration != 0:
                futures.append(executor.submit(parallel_utils.new_mean_process_iteration, iteration, target_file, regs_path, master_regs, load_dir, add_edge, multiple_edges, imp_dir, dataset_id, file_extension, clean))
        
        iter = 0
        chosen_pairs = []
        chosen_pair_filename = os.path.join(experiment_dir, f"DS{dataset_id}_chosen_pairs.csv")
        expression_data = {}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            expr, clean_data, chosen_targets = future.result()
            expression_data['expr'] = expr
            expression_data['clean'] = clean_data
            # ranks, final_ranks, temp_target, f_ext, ds_iter = future.result()
            # #print(chosen_pair, rank)
            # experiment_file_ranks = experiment_file.replace('.csv', f'_iter{ds_iter}.csv')
            # ranks.to_csv(experiment_file_ranks, index=False)
            # #print(final_ranks)
            # for tup in final_ranks:
            #     chosen_pair = (tup[0], tup[1])
            #     add = tup[2]
            #     rank = ranks.index.get_loc(chosen_pair[1]) + 1
            #     if not add:
            #         rank = n_genes - rank
            #     # print(chosen_pair[0], chosen_pair[1], rank, add)
            #     target_val = tup[4]
            #     target_val = ranks.loc[chosen_pair[1]]['mean']
            #     overall_mean = np.mean(ranks['mean'])
            #     chosen_pairs.append([chosen_pair[0], chosen_pair[1], rank, target_val, overall_mean, add, ds_iter])
        #     if ds_iter % 10 != 0:
        #         #print(temp_target)
        #         os.remove(temp_target)
        #         os.remove(os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment_iter{ds_iter}.csv"))
        #         os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_clean{f_ext}.npy"))
        #         os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_noisy{f_ext}.npy"))
        #     os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_expr{f_ext}.npy"))
        #     os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_clean_counts{f_ext}.npy"))
        #     iter += 1
        # chosen_pair_df = pd.DataFrame(chosen_pairs)
        # chosen_pair_df.to_csv(chosen_pair_filename, index=False)
        n_bins, n_sc = 9, 300
        if dataset_id == 1:
            n_genes = 100
        if dataset_id == 2:
            n_genes = 400
        sim = sergio(
            number_genes=n_genes, 
            number_bins = n_bins, 
            number_sc = n_sc,
            # In paper
            noise_params = 1,
            # In paper
            decays=0.8, 
            sampling_state=15, 
            noise_type='dpd')
        
        outliers = [sim.outlier_effect(expression_data['expr'], outlier_prob = 0.01, mean = 5, scale = 1) for i in range(1)]
        expression_data['outliers'] = outliers
        return expression_data, chosen_targets

In [102]:
expr_data, chosen_targets = new_expected_value_experiment(dataset_id=1, last_iteration=50, fixed_edge=False, add_edge=True, multiple_edges=False, clean='noisy')

  0%|          | 0/1 [00:00<?, ?it/s]WARNING:tensorflow:From /Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.11/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
100%|██████████| 1/1 [01:28<00:00, 88.11s/it]


In [105]:
datasets = expr_data['outliers']

reshaped = [np.concatenate(ds, axis = 1) for ds in datasets]
stacked_data = np.stack(reshaped, axis=2)
consensus_data = np.median(stacked_data, axis=2)

np.corrcoef(consensus_data.flatten(), expr_data['clean'].flatten())[0, 1]
# median_expressions = [np.median(np.abs(norm), axis=2) for norm in datasets]

# stacked_data = np.stack(median_expressions)

# consensus_data = np.median(stacked_data, axis=0)

# variability_scores = np.std(consensus_data, axis=0)

# gene_ranks = np.argsort(-variability_scores)

1.0

In [135]:
nGenes_ = 100
nBins_ = 9

def outlier_effect(scData, outlier_prob, mean, scale):
    out_indicator = np.random.binomial(n = 1, p = outlier_prob, size = nGenes_)
    outlierGenesIndx = np.where(out_indicator == 1)[0]
    numOutliers = len(outlierGenesIndx)

    #### generate outlier factors ####
    outFactors = np.random.lognormal(mean = mean, sigma = scale, size = numOutliers)
    ##################################

    scData = np.concatenate(scData, axis = 1)
    for i, gIndx in enumerate(outlierGenesIndx):
        scData[gIndx,:] = scData[gIndx,:] * outFactors[i]

    return np.split(scData, nBins_, axis = 1)

In [136]:
from SERGIO.SERGIO.sergio import sergio

for i in range(100):
    outlier_effect(expr_data['expr'], outlier_prob = 0.01, mean = 5, scale = 1)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [137]:
np.where(expr_data['outliers'] - expr_data['expr'] != 0)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))